# 2D CNN for Text Classification

<h3>Import Libraries</h3>

In [1]:
import sys
!{sys.executable} -m pip install --upgrade keras

    100% |████████████████████████████████| 337kB 3.3MB/s eta 0:00:01
  Using cached six-1.11.0-py2.py3-none-any.whl
Requirement already up-to-date: scipy>=0.14 in /usr/local/lib/python3.6/dist-packages (from keras)
    100% |████████████████████████████████| 12.2MB 126kB/s eta 0:00:01
  Found existing installation: PyYAML 3.11
    Uninstalling PyYAML-3.11:
      Successfully uninstalled PyYAML-3.11
  Found existing installation: six 1.10.0
    Uninstalling six-1.10.0:
      Successfully uninstalled six-1.10.0
  Found existing installation: numpy 1.14.0
    Uninstalling numpy-1.14.0:
      Successfully uninstalled numpy-1.14.0
  Found existing installation: Keras 2.0.6
    Uninstalling Keras-2.0.6:
      Successfully uninstalled Keras-2.0.6


In [2]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.layers import concatenate, CuDNNGRU
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

import sys

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h3>Defining file paths and some basic configuration parameters</h3>

In [3]:
# path = '..'
# EMBEDDING_FILE=path+'/input/glove.840B.300d.txt'
# EMBEDDING_FILE=path+'/input/crawl-300d-2M.vec'
# TRAIN_DATA_FILE=path+'/input/train.csv'
# TEST_DATA_FILE=path+'/input/test.csv'

EMBEDDING_FILE='/public/models/glove/glove.840B.300d.txt'
TRAIN_DATA_FILE='/public/toxic_comments/train.csv'
TEST_DATA_FILE='/public/toxic_comments/test.csv'

MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

<h3>Defining some functions for text processing and cleaning</h3>

In [4]:
def cleanData(text, stemming = False, lemmatize=False):    
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in text.split()])
    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w) for w in text.split()])
    return text


#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)
#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    return(text)

<h3>Read Glove Vectors, Training and Testing Data</h3>

The training data contains a row per comment, with an id, the text of the comment, and 6 different labels that we'll try to predict.
<ul><li>toxic</li><li>severe_toxic</li><li>obscene</li><li>threat</li><li>insult</li><li>identity_hate</li></ul>


<h4>Pre Input Text</h4>

In [5]:
print('Indexing word vectors')

count = 0
embeddings_index = {}
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs.reshape(-1)
    coef = embeddings_index[word]
f.close()

print('Found %d word vectors of glove.' % len(embeddings_index))
emb_mean,emb_std = coef.mean(), coef.std()
print(emb_mean,emb_std)

print('Total %s word vectors.' % len(embeddings_index))

train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

print('Processing text dataset')

train_df['comment_text'] = train_df['comment_text'].map(lambda x: cleanData(x,  stemming = False, 
                                                                            lemmatize=False))
test_df['comment_text'] = test_df['comment_text'].map(lambda x: cleanData(x,  stemming = False, 
                                                                          lemmatize=False))

list_sentences_train = train_df["comment_text"].fillna("NA").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].fillna("NA").values


comments = []
for text in list_sentences_train:
    comments.append(text_to_wordlist(text))
    
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(comments + test_comments)

sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Indexing word vectors
Found 2195895 word vectors of glove.
-0.01444638 0.47249147
Total 2195895 word vectors.
Processing text dataset
Found 292462 unique tokens
Shape of data tensor: (159571, 200)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 200)


<h4>Post Input Text</h4>

In [6]:
data_post = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post', truncating='post')
print('Shape of data tensor:', data_post.shape)
print('Shape of label tensor:', y.shape)

test_data_post = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print('Shape of test_data tensor:', test_data_post.shape)

Shape of data tensor: (159571, 200)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 200)


In [7]:
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 21603


# 2D CNN Network with Pre-Post Input Text

In [23]:
filter_sizes = [1,2,3,5]
num_filters = 32

def get_model(nb_classes, maxlen, feature_output=False, embed_dropout_rate=0, final_dropout_rate=0, embed_l2=1E-6, return_attention=False):
    # define embedding layer that turns word tokens into vectors
    # an activation function is used to bound the values of the embedding
    model_input = Input(shape=(maxlen,), dtype='int32')
    model_input_post = Input(shape=(maxlen,), dtype='int32')
    
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None
    embed = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        embeddings_regularizer=embed_reg,
        trainable=False)
    x1 = embed(model_input)
    x1 = SpatialDropout1D(0.4)(x1)
    x1 = Reshape((200, 300, 1))(x1)
    
    pooled_pre = []
    for i in filter_sizes:
    
        conv_pre = Conv2D(num_filters, kernel_size=(i, 300), kernel_initializer='normal',
                    activation='elu')(x1)
    
        maxpool_pre = MaxPool2D(pool_size=(maxlen - i + 1, 1))(conv_pre)
        avepool_pre = AveragePooling2D(pool_size=(maxlen - i + 1, 1))(conv_pre)
        globalmax_pre = GlobalMaxPooling2D()(conv_pre)
        pooled_pre.append(globalmax_pre)
        
    z1 = Concatenate(axis=1)(pooled_pre)   
    z1 = Dropout(0.2)(z1)
    
    embed_post = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        embeddings_regularizer=embed_reg,
        trainable=False)
    x1_post = embed(model_input_post)
    x1_post = SpatialDropout1D(0.4)(x1_post)
    x1_post = Reshape((200, 300, 1))(x1_post)
    
    pooled_post = []
    for i in filter_sizes:

        conv_post = Conv2D(num_filters, kernel_size=(i, 300), kernel_initializer='normal',
                    activation='elu')(x1_post)
    
        maxpool_post = MaxPool2D(pool_size=(maxlen - i + 1, 1))(conv_post)
        avepool_post = AveragePooling2D(pool_size=(maxlen - i + 1, 1))(conv_post)
        globalmax_post = GlobalMaxPooling2D()(conv_post)
        pooled_post.append(globalmax_post)
        
    z1_post = Concatenate(axis=1)(pooled_post)   
    z1_post = Dropout(0.2)(z1_post)
    
    z = concatenate([z1,z1_post])
    outputs = Dense(6, activation="sigmoid")(z)
    model = Model(inputs=[model_input,model_input_post], outputs=outputs, name="Textcnn2D")
    model.compile(loss='binary_crossentropy',
        optimizer=Adam(lr=1e-3,decay=0),
        metrics=['accuracy'])

    return model

<h3>Model Summary</h3>
<ul><li>Optimizer: Adam</li><li>Learning Rate: 0.001</li><li>Batch Size: 128</li></ul>

<h3>Training the model</h3>

<h4>With 10-fold cross-validation</h4>

In [24]:
from sklearn.metrics import log_loss
import numpy as np

test_predicts_list = []

def train_folds(data,data_post, y, fold_count, batch_size):
    print("Starting to train models...")
    fold_size = len(data) // fold_count
    models = []
    for fold_id in range(0, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size

        if fold_id == fold_size - 1:
            fold_end = len(data)

        print("Fold {0}".format(fold_id))
        
        train_x = np.concatenate([data[:fold_start], data[fold_end:]])
        train_xp = np.concatenate([data_post[:fold_start], data_post[fold_end:]])
        train_y = np.concatenate([y[:fold_start], y[fold_end:]])

        val_x = data[fold_start:fold_end]
        val_xp = data_post[fold_start:fold_end]
        val_y = y[fold_start:fold_end]
        
        file_path="textcnn2D_fold{0}.h5".format(fold_id)
        model = get_model(6, MAX_SEQUENCE_LENGTH, feature_output=False, embed_dropout_rate=0.4, final_dropout_rate=0.4, embed_l2=0, 
                              return_attention=False)
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
        callbacks_list = [checkpoint, early] 

        hist = model.fit([train_x, train_xp], train_y, epochs=15, batch_size=128, shuffle=True, 
                         validation_data=([val_x, val_xp], val_y), callbacks = callbacks_list, verbose=1)
        model.load_weights(file_path)
        best_score = min(hist.history['val_loss'])
        
        print("Fold {0} loss {1}".format(fold_id, best_score))
        print("Predicting validation...")
        val_predicts_path = "textcnn2D_val_predicts{0}.npy".format(fold_id)
        val_predicts = model.predict([val_x, val_xp], batch_size=1024, verbose=1)
        np.save(val_predicts_path, val_predicts)
        
        print("Predicting results...")
        test_predicts_path = "textcnn2D_test_predicts{0}.npy".format(fold_id)
        test_predicts = model.predict([test_data, test_data_post], batch_size=1024, verbose=1)
        test_predicts_list.append(test_predicts)
        np.save(test_predicts_path, test_predicts)

In [25]:
train_folds(data, data_post, y, 10, 128)

Starting to train models...
Fold 0
Train on 143614 samples, validate on 15957 samples
Epoch 1/15
143614/143614 [==============================] - 54s 374us/step - loss: 0.0649 - acc: 0.9781 - val_loss: 0.0509 - val_acc: 0.9810

Epoch 00001: val_loss improved from inf to 0.05094, saving model to textcnn2D_fold0.h5
Epoch 2/15
143614/143614 [==============================] - 51s 354us/step - loss: 0.0492 - acc: 0.9820 - val_loss: 0.0490 - val_acc: 0.9817

Epoch 00002: val_loss improved from 0.05094 to 0.04899, saving model to textcnn2D_fold0.h5
Epoch 3/15
143614/143614 [==============================] - 51s 355us/step - loss: 0.0464 - acc: 0.9826 - val_loss: 0.0429 - val_acc: 0.9836

Epoch 00003: val_loss improved from 0.04899 to 0.04295, saving model to textcnn2D_fold0.h5
Epoch 4/15
143614/143614 [==============================] - 51s 356us/step - loss: 0.0445 - acc: 0.9832 - val_loss: 0.0441 - val_acc: 0.9830

Epoch 00004: val_loss did not improve
Epoch 5/15
143614/143614 [=============

143614/143614 [==============================] - 51s 354us/step - loss: 0.0450 - acc: 0.9830 - val_loss: 0.0431 - val_acc: 0.9836

Epoch 00004: val_loss improved from 0.04452 to 0.04313, saving model to textcnn2D_fold4.h5
Epoch 5/15
143614/143614 [==============================] - 51s 354us/step - loss: 0.0435 - acc: 0.9835 - val_loss: 0.0426 - val_acc: 0.9837

Epoch 00005: val_loss improved from 0.04313 to 0.04258, saving model to textcnn2D_fold4.h5
Epoch 6/15
143614/143614 [==============================] - 51s 356us/step - loss: 0.0426 - acc: 0.9839 - val_loss: 0.0432 - val_acc: 0.9837

Epoch 00006: val_loss did not improve
Epoch 7/15
143614/143614 [==============================] - 51s 357us/step - loss: 0.0414 - acc: 0.9842 - val_loss: 0.0428 - val_acc: 0.9839

Epoch 00007: val_loss did not improve
Epoch 8/15
143614/143614 [==============================] - 51s 357us/step - loss: 0.0406 - acc: 0.9845 - val_loss: 0.0443 - val_acc: 0.9834

Epoch 00008: val_loss did not improve
Fold 

<h3>Predicting for test data and saving results</h3>

<h4>Arithmetic Mean over 10 Folds</h4>

In [26]:
CLASSES = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print(len(test_predicts_list))
test_predicts_am = np.zeros(test_predicts_list[0].shape)

for fold_predict in test_predicts_list:
    test_predicts_am += fold_predict

test_predicts_am = (test_predicts_am / len(test_predicts_list))

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts_am = pd.DataFrame(data=test_predicts_am, columns=CLASSES)
test_predicts_am["id"] = test_ids
test_predicts_am = test_predicts_am[["id"] + CLASSES]
test_predicts_am.to_csv("10fold_2dcnn_am.csv", index=False)

10


<h4>Geometric Mean over 10 Folds</h4>

In [27]:
test_predicts = np.ones(test_predicts_list[0].shape)

for fold_predict in test_predicts_list:
    test_predicts *= fold_predict

test_predicts **= (1. / len(test_predicts_list))

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
test_predicts["id"] = test_ids
test_predicts = test_predicts[["id"] + CLASSES]
test_predicts.to_csv("10fold_2dcnn_gm.csv", index=False)

<h3>ROC AUC Score: <br><br>Private LB: 98.45<br><br>Public LB: 98.52</h3>

<h3>End</h3>